<center>

# **如何运行Heudiconv**

</center>

***

步骤：

1. 搭建运行环境
2. 使用 HeuDiConv 获取 dicominfo.tsv
3. 根据 dicominfo 生成 heuristic.py
4. 使用 heuristic.py 转换并验证 BIDS 是否合规
5. 逐步处理 ses-005、ses-025 等多个 session，重复上述步骤直到数据集完整

注：进行docker pull 不成功，并且是因为网络问题的时候，可参考以下链接：

- https://gitee.com/wanfeng789/docker-hub#docker-hub
- https://www.bilibili.com/video/BV1Zn4y19743/ 
- https://status.1panel.top/status/docker

## 设置运行环境

In [ ]:
# 通过 Docker 拉取 HeuDiConv 镜像
docker pull nipy/heudiconv:latest

## 运行HeuDiConv

首先，查看数据目录结构，如：

```
/home/ss/Downloads/HeuDiConv_walkthrough/Dicom/sub-01/ses-001/SCANS/
   ├── 3/DICOM/*.dcm
   ├── 4/DICOM/*.dcm
   ├── 5/DICOM/*.dcm
   ├── 6/DICOM/*.dcm
   ├── 7/DICOM/*.dcm
   ├── 8/DICOM/*.dcm
   └── 9/DICOM/*.dcm

```

In [ ]:
# 在终端执行以下命令，记得替换路径

docker run --rm -it \
  -v /你的路径/HeuDiConv_walkthrough:/base \
  nipy/heudiconv:latest \
  -d /base/Dicom/sub-{subject}/ses-{session}/SCANS/*/DICOM/*.dcm \
  -o /base/Nifti/ \
  -f convertall \
  -s 01 \
  -ss 001 \
  -c none \
  --overwrite

运行后，会在 Nifti/ 下生成一个隐藏目录 .heudiconv，包含 dicominfo_ses-001.tsv 文件。

## 提取 dicominfo.tsv 并编写 heuristic.py

将生成的 dicominfo_ses-001.tsv 文件复制出来（例如放在项目根目录）：

In [ ]:
# 记得替换路径
cp /你的路径/HeuDiConv_walkthrough/Nifti/.heudiconv/01/info/dicominfo_ses-001.tsv /你的路径/HeuDiConv_walkthrough/


打开这个文件，查看其中多个参数字段（如 SeriesDescription、ProtocolName 等），用于生成你的 heuristic.py 文件。

若使用本教程的示例教程，可参考本教程的 heuristic.py 

注：本教程的heuristic.py放在 `HeuDiConv_walkthrough` 文件夹的根目录下

In [ ]:
def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    return template, outtype, annotation_classes

def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where"""
    info = {}

    
    t1w = create_key('sub-{subject}/{session}/anat/sub-{subject}_{session}_T1w')

    info[t1w] = []

    for s in seqinfo:
        desc = s.series_description.lower()
        if "t1" in desc or "mprage" in desc:
            info[t1w].append(s.series_id)

    return info


## 使用 heuristic.py 再次运行 HeuDiConv 并验证

使用包含自定义规则的 `heuristic.py` 再次运行转换命令，基本结构与上次相同，但加入 `-f heuristic.py` 参数。

In [ ]:
sudo docker run --rm -it \
-v /home/ss/Downloads/HeuDiConv_walkthrough:/base \
nipy/heudiconv:latest \
-d /base/Dicom/sub-{subject}/ses-{session}/SCANS/*/DICOM/*.dcm \
-o /base/Nifti/ \
-f /base/heuristic.py \
-s 01 \
-ss 001 \
-c dcm2niix \
--overwrite


完成后，可使用 BIDS Validator 检查生成的数据集是否合法、结构是否正确。

注：我们仅示范了一个session的数据转换过程，对于其他的session 可以同样执行上述代码，修改 docker 执行命令中的对应参数即可。